# **Pan-sharpening**

เทคนิค Pan-sharpening เป็นเทคนิคที่ใช้กับภาพถ่ายดาวเทียม ที่มี Panchromatic band เช่น THEOS-1, LANDSAT-8, LANDSAT-9 เพื่อให้มีรายละเอียดและมีประโยชน์มากขึ้น โดย Lab นี้เราจะทดลองกับข้อมูลดาวเทียม Landsat-9

ข้อมูลถ่ายที่เป็น Multi-Spectral มักจะมีรายละเอียดภาพไม่ค่อยดีนัก เมื่อเทียบกับข้อมูล Panchromatics  ดังนั้น การปรับความคมชัดของภาพช่วยได้โดยการรวมภาพเหล่านี้เข้ากับภาพขาวดำพิเศษที่มีรายละเอียดมากขึ้น ภาพขาวดำที่เรียกว่าภาพแพนโครมาติก มีรายละเอียดในระดับที่สูงกว่า แต่ไม่มีสี เราต้องการใช้รายละเอียดจากภาพแพนโครมาติกและสีจากภาพ Landsat-9 เพื่อสร้างภาพสีที่มีรายละเอียดสูงใหม่

ในการดำเนินการใน Lab นี้ นักศึกษา จำเป็นต้องตรวจสอบให้แน่ใจว่าข้อมูลทั้ง Panchromatic และ Multispectral ของ Landsat-9 อยู่ในตำแหน่งที่สมบูรณ์แบบ เพื่อให้เข้ากันได้อย่างลงตัว

มีหลายวิธีในการรวมภาพเหล่านี้ เช่น Brovey Transform, IHS Transform และ PCA วิธีการเหล่านี้ทำให้แน่ใจว่าภาพใหม่จะคงสีจาก Landsat-9 แต่ยังได้รับรายละเอียดเพิ่มเติมจากภาพแพนโครมาติกด้วย


เมื่อเราทำ Pan-sharpening เสร็จแล้ว เราก็จะได้ภาพใหม่ที่เป็นภาพสีที่มีรายละเอียดสูงขึ้น  โดยมีสีทั้งหมดจาก Landsat-9 และความคมชัดพิเศษจากภาพแบบแพนโครมาติก รูปภาพใหม่นี้สามารถนำไปใช้ได้หลายอย่าง เช่น ศึกษาพื้นดิน ค้นหาการเปลี่ยนแปลง หรือเพียงแค่ดูรายละเอียดเพิ่มเติมเกี่ยวกับโลก ดังนั้น การปรับความคมชัดของภาพเป็นวิธีหนึ่งในการทำให้ภาพจากดาวเทียมมีประโยชน์มากขึ้นสำหรับงานสำคัญทุกประเภท

In [1]:
# ทำการ Authenticate และ initialize Earth Engine
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-khehathancencira') #อย่าลืมเปลี่ยนชื่อโปรเจคของตัวเอง

In [2]:
# กำหนดพื้นที่สนใจ
geometry = ee.Geometry.Point([98.95799098999555, 18.84423947416328])

# เรียกภาพ L9 ตัวอย่างแล้วดึง RGB และ Pan ออกมา
image = (ee.ImageCollection("LANDSAT/LC09/C02/T1_TOA")
         .filterDate('2022-01-01', '2022-03-30')
         .filterBounds(geometry)
         .sort('CLOUD_COVER')
         .first())

In [3]:
# ทำการ Pan-Sharp
rgb = image.select('B4', 'B3', 'B2')
pan = image.select('B8')

# แปลงเป็น HSV, สลับในแถบ PAN และแปลงกลับเป็น RGB
huesat = rgb.rgbToHsv().select('hue', 'saturation')
upres = ee.Image.cat([huesat, pan]).hsvToRgb()

In [4]:
# สร้างแผนที่
Map = geemap.Map(center=[18.84423947416328, 98.95799098999555], zoom=14)

# แสดง เลเยอร์ ก่อนและหลังโดยใช้พารามิเตอร์ vis เดียวกัน
Map.addLayer(rgb, {'max': 0.28}, 'Original')
Map.addLayer(pan, {'max': 0.28}, 'Pan')
Map.addLayer(upres, {'max': 0.28}, 'Pansharpened')
Map


Map(center=[18.84423947416328, 98.95799098999555], controls=(WidgetControl(options=['position', 'transparent_b…

คำถาม
ข้อเพื่อทดสอบความเข้าใจของนักศึกษาเกี่ยวกับเทคนิคการทำ Pan-sharpening ด้วยภาพ Landsat-9 จงตอบคำถามต่อไปนี้

1. อะไรคือเป้าหมายหลักของการปรับความคมชัดของภาพในการสำรวจระยะไกล โดยเฉพาะเมื่อใช้ภาพ Landsat-9 อธิบายว่าเหตุใดจึงมีความสำคัญในการประมวลผลภาพ

ตอบ เพื่อเพิ่มความละเอียดเชิงพื้นที่ และเพิ่มความคมชัดให้  Landsat-9 ที่เหมาะกับการแปลด้วยสายตาที่ต้องอาศัยการมองเห็นวัตถุชัดเจน

2. อธิบายขั้นตอนสำคัญที่เกี่ยวข้องกับการปรับความคมชัดด้วนเทคนิค Pan-sharpening ตั้งแต่การรับข้อมูลไปจนถึงการสร้างภาพที่ปรับความคมชัด เทคนิค Pan-sharpening มี กระบวนการสุ่มตัวอย่างใหม่ช่วยจัดแนวภาพหลายสเปกตรัมและภาพแพนโครมาติกอย่างไร

ตอบ Pan-sharpening คือ การรวมภาพถ่ายจากดาวเทียมความละเอียดเชิงพื้นที่สูงขาวดำ (PAN) กับภาพสีความละเอียดต่ำ (Multispectral) เพื่อสร้างภาพสีที่ความละเอียดสูง

ขั้นตอนการทำ Pan-sharpening
1. มีข้อมูลภาพถ่ายจากดาวเทียม 2 ภาพ ได้แก่ ภาพ PAN มี pixel size 15 เมตร กับภาพ Multispectral มี pixel size 30 เมตร
2. การจัดแนวและปรับสุ่มตัวอย่างใหม่ โดยกระบวนการสุ่มตัวอย่างใหม่ (Resampling) เช่น Bilinear หรือ Cubic Convolution เพื่อปรับขยายขนาดภาพถ่ายจากดาวเทียม โดยภาพสี Multispectral ให้มีพิกเซลตรงกันและมีความละเอียดเชิงพื้นที่เท่ากับภาพ PAN ก่อนรวมภาพ
3. การแปลงข้อมูล (Transformation): แปลงข้อมูลภาพMultispectral จากระบบสีปกติ  ไปเป็นสีที่แยกส่วนความสว่าง ออกจากสี (Hue/Saturation) เช่น การแปลง IHS หรือ PCA เพื่อให้สลับรายละเอียดความสว่างได้
4. การรวมภาพ (Substitution/Fusion): แทนที่ส่วนประกอบความเข้มแสง (Intensity หรือ V) ของภาพ Multispectral ด้วยค่าความสว่างจากภาพ PAN ที่มีความละเอียดสูงกว่า
5. การแปลงกลับ (Inverse Transform): แปลงข้อมูลจากระบบที่แปลงไว้ กลับสู่ระบบสีเดิม (RGB) จะได้ผลลัพธ์เป็นภาพสีที่มีความคมชัดสูง 15 เมตร